# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

дополнения: *Ян Пиле, НИУ ВШЭ*

### Работа с `selenium`: продолжение

Сегодня мы продолжим работать над задачей, поставленной на прошлом занятии ‒ выгрузка адресов всех участковых избирательных комиссий Ивановской области. Сначала загрузим все необходимые для работы библиотеки:

* `selenium` ‒ для автоматизации работы в браузере
* `re` ‒ для поиска адреса на странице с помощью регулярных выражений 
* `time` ‒ для добавления задержки
* `pandas` ‒ для сохранения результатов в датафрейм

In [1]:
from selenium import webdriver as wb
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
# driver = wb.Firefox()

# driver.implicitly_wait(2)

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 106.0.5249
[WDM] - Get LATEST chromedriver version for 106.0.5249 google-chrome
[WDM] - Driver [/Users/Dell/.wdm/drivers/chromedriver/mac64/106.0.5249.61/chromedriver] found in cache
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
import re
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
driver.get("http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/")

fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")
fld.click()

/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/3525716967.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")


Теперь напишем функцию `get_uik_address()`, которая принимает на вход два аргумента, номер участка и регион, и возвращает строку с адресом. Для этого в тело функции скопируем код с прошлого занятия:

In [5]:
def get_uik_address(n_uik, reg):
    
    driver.get("http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/")

#     fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")
#     fld.click()

#     sleep(1.5)

    fld = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
    fld.click()

    sleep(1.5)

    fld = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
    fld.click()

    sleep(1.5)

    reg_field = driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
    reg_field.send_keys(reg)

    sleep(1.5)

    reg_field.send_keys(Keys.RETURN)

    sleep(1.5)

    uik_field = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[2]/div/input')
    uik_field.click()
    uik_field.clear()
    uik_field.send_keys(n_uik)

    sleep(1.5)

    uik_field.send_keys(Keys.RETURN)
    
    sleep(1.5)
    
    my_page = BeautifulSoup(driver.page_source)
    p = filter(lambda x: x not in ('',None),[i.text for i in my_page.find_all('span',{'class':'address'})])
    p = next(p)
    
    if p is None or p=='':
        p = 'Адреса не нашлось'
        
    return p

Теперь попробуем взять несколько номеров участков и посмотреть, что получается в цикле. Только давайте перестрахуемся ‒ напишем выражение с исключением, чтобы в случае, если страница не содержит адреса или загружается некорректно, наш код не ломался и не происходило выхода из цикла. В случае, если все хорошо (адрес есть), Python будет его сохранять («ветка» c `try`), в случае, если все плохо (адреса нет ни в каком виде), Python будет записывать вместо него пустую строку (ветка с `except`) и двигаться дальше. 

In [6]:
uiks = range(240, 245)

In [7]:
get_uik_address(244, "Ивановская область")

/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  fld = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  fld = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  reg_field = driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:29: DeprecationWar

'155800, Ивановская область, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"'

In [8]:
addresses = []

for u in uiks:
    try:
        address = get_uik_address(u, "Ивановская область")
    except:
        address = ""
    addresses.append(address)
    print(u, address)

/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  fld = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  fld = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  reg_field = driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
/var/folders/4l/wvvj0csx3cd5ksl4vg3g_knm0000gn/T/ipykernel_42748/4207432635.py:29: DeprecationWar

240 
241 155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6
242 155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"
243 
244 


Работает! Создадим список со всеми номерами избирательных участков Ивановской области:

In [9]:
addresses

['',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"',
 '',
 '']

**Важно:** периодически открывайте окно браузера, в котором Python ищет избирательные участки! Это не только приятно (смотреть, как в полях для поиска все заполняется без нашего участия), но и полезно: так можно заметить, если что-то пошло не так. 

Создадим датафрейм из словаря, ключами которого служат названия столбцов таблицы, а значениями – списки элементов этих столбцов.

In [10]:
df = pd.DataFrame({'uik': uiks, 'address': addresses})

In [11]:
df.head()

,uik,address
0,240,
1,241,"155330, Ивановская область, городской округ Ви..."
2,242,"155330, Ивановская область, городской округ Ви..."
3,243,
4,244,


In [12]:
list(df.address)[0:10]

['',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"',
 '',
 '']

Сохраним таблицу в csv-файл:

In [13]:
df.to_csv('Ivanovo.csv', index=False)

In [14]:
driver.close()